# LU Decomposition and Condition Number

CBE 20258. Numerical and Statistical Analysis. Spring 2020.

&#169; University of Notre Dame

## Learning Objectives

After studying this notebook, completing the activities, and asking questions in class, you should be able to:
* Explain to a classmate how Gaussian elimination and LU factorization are similar and different
* In Python, use LU factorization to solve a linear system of equations
* For LU factorization with partial pivoting, explain the following (same as Gaussian elimination last class):
    * Why is it important? What are the benefits?
    * How does the algorithm work? What are the basic steps?
    * What are the limitations? What cases will partial pivoting not work?
* Interpret the rank of a matrix. What does it mean about the corresponding linear system?
* Propagate uncertainty through the solution of a linear system using condition number.

In [1]:
import numpy as np

## Motivation: Solving the "Same" System Multiple Times

In practice, you often want to solve the same linear system multiple times with different right-hand side vectors $\vec{b}$. For example, imagine resolving the same mass balance problem with different inlet and outlet compositions.

From the previous class, we know how to use Gaussian elimination to solve a linear system. But with Gaussian elimination, we need to redo the entire algorithm separately for each new instance of $\vec{b}$. Is there an algorithm that lets us reuse some of our work? Spoiler alert: yes, and it is called LU factorization.

## LU Factorization

LU factorization writes a matrix $\mathbf{A}$ as the product of a lower triangular matrix (that is a matrix with nonzero elements on the diagonal and below) times an upper triangular matrix (that is a matrix with nonzero elements above the diagonal). In other words, $\mathbf{A} = \mathbf{LU},$ where $$\mathbf{L} = \begin{pmatrix} * & 0 & \dots\\ * & * & 0 & \dots\\ * & * & * &0 & \dots\\ \vdots & \vdots & \vdots & \ddots\\ * & * & * & \dots & * \end{pmatrix},$$ and $$\mathbf{U} = \begin{pmatrix} * & * & * & \dots & * \\ 
0 & * & * & \dots & * \\ 
0 & 0 & \ddots & \dots & *\\ 
\vdots & \vdots & \vdots & \ddots\\
0 & \dots & &0 & *\end{pmatrix},$$ where the $*$ denote a potentially nonzero matrix element.  

The question remains; how we find the entries in the matrices in the LU factorization?  To see this, let's look at a matrix we used in last class: $$\begin{pmatrix} 3 & 2 & 1\\ -1 & 4 & 5\\ 2& -8 & 10\end{pmatrix},$$ we want to find the matrix LU such that 
$$\begin{pmatrix} l_{11} & 0 & 0\\ l_{21} & l_{22} & 0\\ l_{31}& l_{32} & l_{33}\end{pmatrix} \begin{pmatrix} u_{11} & u_{12} & u_{13}\\ 0 & u_{22} & u_{23}\\ 0& 0 & u_{33}\end{pmatrix}=\begin{pmatrix} 3 & 2 & 1\\ -1 & 4 & 5\\ 2& -8 & 10\end{pmatrix}.$$

<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
<b>Home Activity</b>: Please read through the explanation below. Think of at least one question to ask in class.
</div>

It's not exactly clear right now, but we actually have a few choices at this point.  The first choice we have is which diagonal, either $\mathbf{L}$ or $\mathbf{U}$, we want to be all ones.  We will choose to have all the $l_{ii} = 1$, that is the $\mathbf{L}$ matrix has a diagonal of all 1's. (This means we are choosing Doolittle's method for LU factorization.) Performing the matrix multiplication we get 9 equations and 9 unknowns for the factorization. The product of the first column of $\mathbf{U}$ by the first column of $\mathbf{L}$ gives 
$$ u_{11} = 3\\ l_{21} u_{11} = -1\\ l_{31} u_{11} = 2.$$ Which gives us
$$u_{11} = 3,\\ l_{21} = -\frac{1}{3},\\ l_{31} = \frac{2}{3}.$$ Notice that the solutions below the diagaonal, the $l_{21}$ and $l_{31}$, are the factors we used in our Gaussian elimination algorithm to remove the first column in the second and third equations. And the $u_{11}$ value is that found in the same position in our Gaussian-eliminated matrix. 

Continuing on, the equations from the second column are
$$ u_{12} = 2,\\ u_{12}l_{21} + u_{22}  = 4,\\ u_{12} l_{31} + u_{22} l_{32} = -8,$$ the solutions to these equations are 
$$u_{12} = 2,\\ u_{22} =  4\frac{2}{3}, \\ l_{32} = -2.$$ You probably don't remember, because I didn't, but the value of $l_{32}$ is the value of the factor used to eliminate the second column below the diagonal in the Gaussian elimination example from the last notebook. Finally, the last three equations are 
$$ u_{13} = 1,\\u_{13} l_{21} + u_{23} = 5,\\ u_{13} l_{31} + u_{23}l_{32} + u_{33} = 10.$$ The solutions to these equations are
$$u_{13} = 1, \\u_{23} = 5\frac{1}{3},\\ u_{33} = 20.$$ These are precisely the values of the third column in or Gaussian elimination example. 

In other words our factorization looks like:
$$\underbrace{\begin{pmatrix} 1 & 0 & 0\\ -\frac{1}{3} & 1& 0\\ \frac{2}{3} & -2 & 1\end{pmatrix}}_{\mathbf{L}} \underbrace{\begin{pmatrix} 3 & 2 & 1\\ 0 & 4\frac{2}{3} & 5\frac{1}{3}\\ 0& 0 & 20\end{pmatrix}}_{\mathbf{U}}=\underbrace{\begin{pmatrix} 3 & 2 & 1\\ -1 & 4 & 5\\ 2& -8 & 10\end{pmatrix}}_{\mathbf{A}}.$$

<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
    <b>Home Activity</b>: Let's verify the above equation is correct. Create numpy arrays <tt>L</tt> and <tt>U</tt> for the example above. Then multiple matrices <tt>L</tt> and <tt>U</tt> and store the results in <tt>A</tt>.
</div>

In [2]:
### BEGIN SOLUTION
L = np.array([[1, 0, 0], [-1/3, 1, 0], [2/3, -2, 1]])
U = np.array([[3, 2, 1], [0, 4 + 2/3, 5 + 1/3], [0, 0, 20]])
print("L = \n",L,"\n")
print("U = \n",U,"\n")

A = L @ U
print("A = \n",A,"\n")
### END SOLUTION

L = 
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.66666667 -2.          1.        ]] 

U = 
 [[ 3.          2.          1.        ]
 [ 0.          4.66666667  5.33333333]
 [ 0.          0.         20.        ]] 

A = 
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]] 



In [3]:
### BEGIN HIDDEN TESTS

L_ans = np.array([[1, 0, 0], [-1/3, 1, 0], [2/3, -2, 1]])
U_ans = np.array([[3, 2, 1], [0, 4 + 2/3, 5 + 1/3], [0, 0, 20]])
A_ans = np.array([[ 3.,  2.,  1.], [-1.,  4.,  5.], [ 2., -8., 10.]])

assert np.shape(L) == (3,3), "L should be a 3x3 numpy array"
assert np.shape(U) == (3,3), "U should be a 3x3 numpy array"
assert np.shape(A) == (3,3), "A should be a 3x3 numpy array"

for i in range(3):
    for j in range(3):
        assert np.abs(L_ans[i,j] - L[i,j]) < 1E-6, "Check the ("+str(i)+","+str(j)+") element in L"
        assert np.abs(U_ans[i,j] - U[i,j]) < 1E-6, "Check the ("+str(i)+","+str(j)+") element in U"

assert np.linalg.norm(A - A_ans) < 1E-6, "Check your A matrix"

### END HIDDEN TESTS

Some observations:
1. The upper triangular matrix is the same as the matrix we received after doing Gaussian elimination
2. The non-zero and non-diagonal elements of the lower triangular matrix are the factors we used to arrive at our Gaussian matrix.

This suggests that we can reformulate our Gaussian elimination example to give us an LU factorization.

<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
<b>Home Activity</b>: Write down questions you can ask in class about the example above.
</div>

Record your question(s) here.

<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
<b>Class Activity</b>: Walk through the LU factorization example together.
</div>

### Our First Function for LU Factorization

We will now generalize the LU factorization algorithm into a Python function. 

In [6]:
import numpy as np
def LU_factor1(A,LOUD=True):
    '''Compute LU (Doolitte) decomposition of A
    
    Args:
        A: N by N array
    
    Returns:
        L: N by N array
        U: N by N array
    
    Side Effect:
        Does not modify A
    '''
    
    # Check dimensions
    [Nrow, Ncol] = A.shape
    assert Nrow == Ncol
    N = Nrow
    
    # Allocate L as identify matrix
    L = np.eye(N)
    
    # Copy A into U
    U = A.copy()
    
    # Loop over columns
    for c in range(0,N):
    
        # Loop over rows
        for r in range(c+1,N):
        
            # Extract row to modify
            mod_row = U[r,:]
            
            # Calculate factor (just like Gaussian Elimination)
            factor = mod_row[c]/U[c,c]
            
            # Save factor into L
            L[r,c] = factor
            
            # Perform elimination, save into U
            mod_row -= factor*U[c]
            
            #mod_row = U[r-1,:]
            
            if LOUD:
                print("After working on column",c,"and row",r)
                print("L = \n",L)
                print("U = \n",U)
                print("\n")
    return L,U
            
#let's try it on a 3 x 3 to start
A = np.array([(3.0,2,1),(-1,4,5),(2,-8,10)])
print("The original matrix is\n",A)
L,U = LU_factor1(A)
#print("The LU factored A is\n",A) 
    
print("L * U = \n")
print(L,"\n*\n",U,"\n=\n",L.dot(U))

The original matrix is
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]]
After working on column 0 and row 1
L = 
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.          0.          1.        ]]
U = 
 [[ 3.          2.          1.        ]
 [ 0.          4.66666667  5.33333333]
 [ 2.         -8.         10.        ]]


After working on column 0 and row 2
L = 
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.66666667  0.          1.        ]]
U = 
 [[ 3.          2.          1.        ]
 [ 0.          4.66666667  5.33333333]
 [ 0.         -9.33333333  9.33333333]]


After working on column 1 and row 2
L = 
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.66666667 -2.          1.        ]]
U = 
 [[ 3.          2.          1.        ]
 [ 0.          4.66666667  5.33333333]
 [ 0.          0.         20.        ]]


L * U = 

[[ 1.          0.          0.        ]
 [-0.33333333  1.      

We see that our function gives the same `L` and `U` as before. At the start of class, we will work through the LU decomposition example on the board together for this example. We will then revist the function above.

<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
    <b>Home Activity</b>: Perform LU decomposition for matrix <tt>A1</tt>, which is defined below. Store the results in variables <tt>L1</tt> and <tt>U1</tt>.
</div>

In [7]:
A1 = np.array([[1., 2, 3],[-1, 2, 1], [1, 2, 0]])
print("A1 = \n",A1,"\n")

### BEGIN SOLUTION
L1, U1 = LU_factor1(A1)
### END SOLUTION

A1 = 
 [[ 1.  2.  3.]
 [-1.  2.  1.]
 [ 1.  2.  0.]] 

After working on column 0 and row 1
L = 
 [[ 1.  0.  0.]
 [-1.  1.  0.]
 [ 0.  0.  1.]]
U = 
 [[1. 2. 3.]
 [0. 4. 4.]
 [1. 2. 0.]]


After working on column 0 and row 2
L = 
 [[ 1.  0.  0.]
 [-1.  1.  0.]
 [ 1.  0.  1.]]
U = 
 [[ 1.  2.  3.]
 [ 0.  4.  4.]
 [ 0.  0. -3.]]


After working on column 1 and row 2
L = 
 [[ 1.  0.  0.]
 [-1.  1.  0.]
 [ 1.  0.  1.]]
U = 
 [[ 1.  2.  3.]
 [ 0.  4.  4.]
 [ 0.  0. -3.]]




In [8]:
### BEGIN HIDDEN TESTS
L1_ans = np.array([[ 1.,  0.,  0.], [-1.,  1.,  0.], [ 1.,  0.,  1.]])
U1_ans = np.array([[ 1.,  2.,  3.], [ 0.,  4.,  4.],[ 0.,  0., -3.]])
assert np.shape(L1) == (3,3), "L1 should be a 3x3 matrix"
assert np.shape(U1) == (3,3), "U1 should be a 3x3 matrix"

assert np.linalg.norm(L1 - L1_ans) < 1E-4, "L1 is not correct. Check you are passing in the correct A matrix."
assert np.linalg.norm(U1 - U1_ans) < 1E-4, "U1 is not correct. Check you are passing in the correct A matrix."
### END HIDDEN TESTS

<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
<b>Home Activity</b>: Spend 5 honest minutes reading through the function <tt>LU_factor1</tt>. Write down at least two questions you have right now.
</div>

My questions while inspecting the code below:
1. 
2. 

<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
<b>Class Activity</b>: Revist your list of questions about <tt>LU_factor1</tt> with your neighbor. We'll then regroup and discuss as a class.
</div>

### A More Memory Efficient Version

Our function `LU_factor1` makes a copy of `A` to initialize `U`. This is nice because it does not overwrite the original `A`. But imagine if `A` was a 10 million by 10 million matrix. Making a copy would be expensive and require a lot of memory.
    
Instead, the function below also performs LU factorization. But instead of returning ``L`` and ``U``, it overwrites `A` with the factorization result. This avoids copying large matrices.

In [9]:
import numpy as np
def LU_factor(A):
    """Factor in place A into L*U=A. 
    The lower triangular parts of A
    are the L matrix.  
    The L has implied ones on the diagonal.
    Args:
        A: N by N array
    Side Effects:
        A is factored in place.
    """
    [Nrow, Ncol] = A.shape
    assert Nrow == Ncol
    N = Nrow
    for column in range(0,N):
        for row in range(column+1,N):
            mod_row = A[row]
            factor = mod_row[column]/A[column,column]
            mod_row = mod_row - factor*A[column,:]
            #put the factor in the correct place in the modified row
            mod_row[column] = factor
            #only take the part of the modified row we need
            mod_row = mod_row[column:N]
            A[row,column:N] = mod_row
    return

#let's try it on a 3 x 3 to start
A = np.array([(3.0,2,1),(-1,4,5),(2,-8,10)])
print("The original matrix is\n",A)
LU_factor(A)
print("The LU factored A is\n",A)

The original matrix is
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]]
The LU factored A is
 [[ 3.          2.          1.        ]
 [-0.33333333  4.66666667  5.33333333]
 [ 0.66666667 -2.         20.        ]]


Notice how our matrix `A` has changed:

In [10]:
print(A)

[[ 3.          2.          1.        ]
 [-0.33333333  4.66666667  5.33333333]
 [ 0.66666667 -2.         20.        ]]


<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
    <b>Home Activity</b>: Use <tt>LU_factor</tt> for the example below. Your matrix <tt>A1</tt> should change.
</div>

In [11]:
A1 = np.array([[1., 2, 3],[-1, 2, 1], [1, 2, 0]])
print("A1 (original) = \n",A1,"\n")

### BEGIN SOLUTION
LU_factor(A1)
### END SOLUTION

print("A1 (factored) = \n",A1,"\n")

A1 (original) = 
 [[ 1.  2.  3.]
 [-1.  2.  1.]
 [ 1.  2.  0.]] 

A1 (factored) = 
 [[ 1.  2.  3.]
 [-1.  4.  4.]
 [ 1.  0. -3.]] 



In [12]:
### BEGIN HIDDEN TESTS
A1_ans_factored = np.array([[1., 2., 3.],[-1., 4., 4.], [1., 0., -3.]])
assert np.shape(A1) == (3,3), "A1 should be a 3x3 matrix"
assert np.linalg.norm(A1 - A1_ans_factored) < 1E-4, "A1 is not correct. Check you are passing in the correct A matrix and using LU_factor not LU_factor1."
### END HIDDEN TESTS

## Solving Linear Systems

This algorithm gives us the same thing our onerous, by-hand procedure did. The question we have not answered is how we can take an LU factored matrix and get the solution to the system $\mathbf{A}\vec{x} = \vec{b}$.

Let's start by substituting $\mathbf{L U} = \mathbf{A}$ into our original equation:

$$\mathbf{A}\vec{x} = \mathbf{LU}\vec{x} =  \vec{b}$$

Now let's create a new intermediate variable $\vec{y} = \mathbf{U} \vec{x}$. If we substitute $\vec{y}$ into the $\mathbf{LU}\vec{x} =  \vec{b}$, we get:

$$\mathbf{L}\vec{y} = \vec{b}$$

<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
    <b>Home Activity</b>: Compute $\vec{y}$ for the example below by hand or with Python. Store the results in a numpy array <tt>y</tt>.
</div>

$$\mathbf{L} = \begin{pmatrix} 1 & 0 & 0\\ -\frac{1}{3} & 1& 0\\ \frac{2}{3} & -2 & 1\end{pmatrix}, \quad \vec{b} = \begin{pmatrix}6 \\ 8 \\ 4 \end{pmatrix}
$$

In [13]:
### BEGIN SOLUTION
y = np.array([6, 10, 20])
### END SOLUTION

In [14]:
### BEGIN HIDDEN TESTS
assert len(y) == 3, "y should have 3 elements"
assert np.abs(y[0] - 6) < 1E-10, "Check y[0]"
assert np.abs(y[1] - 10) < 1E-10, "Check y[1]"
assert np.abs(y[2] - 20) < 1E-10, "Check y[2]"
### END HIDDEN TESTS

You may have noticed in the example, we can easily compute $\vec{y}$ because $\mathbf{L}$ is a lower triangular matrix. We start at the top to calculate the first element of $\vec{y}$. We then move to the next row of $\mathbf{L}$, substitute and compute the next element of $\vec{y}$. We call this procedure forward substitution because we start at the top.

We now have $\vec{y}$. We are half way there! We just need to solve the linear system $\mathbf{U}\vec{x} = \vec{y}$.

<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
<b>Home Activity</b>: Compute $\vec{x}$ for the example below either by hand or with Python. Store your answer in the numpy array <tt>x</tt>.
</div>

$$\mathbf{U} = \begin{pmatrix} 3 & 2 & 1\\ 0 & 4\frac{2}{3} & 5\frac{1}{3}\\ 0& 0 & 20\end{pmatrix}, \quad \vec{y} = \begin{pmatrix}6 \\ 10 \\ 20 \end{pmatrix}
$$

In [15]:
### BEGIN SOLUTION
x = np.array([1, 1, 1])
### END SOLUTION

In [16]:
### BEGIN HIDDEN TESTS
assert len(x) == 3, "x should have 3 elements"
assert np.abs(x[0] - 1) < 1E-10, "Check x[0]"
assert np.abs(x[1] - 1) < 1E-10, "Check x[1]"
assert np.abs(x[2] - 1) < 1E-10, "Check x[2]"
### END HIDDEN TESTS

Notice again this calculation is easy because $\mathbf{U}$ is upper triangular. We start at the bottom and calculate the last element of $\vec{x}$. We then move up a row, substitute, and calculate the second to last element of $\vec{x}$. This is called backward substitution and is the same procedure we used with Gaussian elimination last class.

### Our First Function for Forward and Backward Solves

Let's return to our first factorization function `LU_factor` which returned a distinct ``L`` and ``U``. We will create Python code to generalize the forward and backward substitution steps to solve the linear system.


In [17]:
def LU_solve1(L,U,b,LOUD=True):
    '''Solve L*U*x = b for x where L and U
    are lower and upper triangular matrices and
    b is RHS vector.
    
    Args:
        L: N by N lower triangular matrix with 1 on diagonal
        U: N by N upper triangular matrix
        b: N by 1 vector
        
    Returns
        x: N by 1 solution to linear system
    '''
    
    # Check dimensions
    [Nrow_L, Ncol_L] = L.shape
    assert Nrow_L == Ncol_L
    [Nrow_U, Ncol_U] = U.shape
    assert Nrow_U == Ncol_U
    assert Nrow_U == Nrow_L
    N = Nrow_U
    assert N == b.size
    
    # forward solve
    y = np.zeros(N)
    if LOUD:
        print("\nForward solve...")
    for r in range(0,N): # first for loop
        RHS = b[r]
        
        # loop over some of the columns to evaluated summation
        for c in range(0,r):
            RHS -= y[c]*L[r,c]
        
        if LOUD:
            print("y[",r,"]=",RHS)
        
        y[r] = RHS
    
    # backward solve
    x = np.zeros(N)
    if LOUD:
        print("\nBackward solve...")
    for r in range(N-1,-1,-1): # second for loop
        RHS = y[r]
        
        # loop over some of the columns to evaluated summation
        for c in range(r+1,N):
            RHS -= x[c]*U[r,c]
        
        if LOUD:
            print("x[",r,"]=",RHS/U[r,r])
        x[r] = RHS/U[r,r]
    
    return x

Let's take a minute to look at the first and second for loops in `LU_solve1`.

In [18]:
N = 5

# loop 1
print("Loop 1...")
for i in range(0,N):
    print(i)
    
# loop 2
print("\nLoop 2...")
for i in range(N-1,-1,-1):
    print(i)

Loop 1...
0
1
2
3
4

Loop 2...
4
3
2
1
0


<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
    <b>Home Activity</b>: Why does the first for loop iterate over <tt>range(0,N)</tt> and the second for loop iterate over <tt>range(N-1,-1,-1)</tt> in <tt>LU_solve1</tt>? Write a sentence below.
</div>

<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
<b>Class Activity</b>: Discuss the code above with a partner. We'll then regroup as a class.
</div>

And let's test with the example above.

In [19]:
A = np.array([(3.0,2,1),(-1,4,5),(2,-8,10)])
b = np.array([6,8,4])
print("The original matrix is\n",A)

L,U = LU_factor1(A,False)
print("L = \n",L)
print("\nU= \n",U)

x = LU_solve1(L,U,b)

The original matrix is
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]]
L = 
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [ 0.66666667 -2.          1.        ]]

U= 
 [[ 3.          2.          1.        ]
 [ 0.          4.66666667  5.33333333]
 [ 0.          0.         20.        ]]

Forward solve...
y[ 0 ]= 6
y[ 1 ]= 10.0
y[ 2 ]= 20.0

Backward solve...
x[ 2 ]= 1.0
x[ 1 ]= 1.0
x[ 0 ]= 1.0


And now verify the solution.

In [20]:
print("A*x =",A.dot(x))
print("b =",b)

A*x = [6. 8. 4.]
b = [6 8 4]


### Another Implementation (Optional)

Now let's consider the case where ``L`` and ``U`` are stored in the same matrix. This complements our more memory efficient function `LU_factor`.

In [21]:
def LU_solve(A,b):
    """Take a LU factorized matrix and 
    solve it for RHS b
    Args:
        A: N by N array that has been LU factored with
        assumed 1's on the diagonal of the L matrix
        b: N by 1 array of righthand side
    Returns:
        x: N by 1 array of solutions
    """
    [Nrow, Ncol] = A.shape
    assert Nrow == Ncol
    N = Nrow
    
    x = np.zeros(N)
    #temporary vector for L^-1 b
    y = np.zeros(N)
    #forward solve
    for row in range(N):
        RHS = b[row]
        for column in range(0,row):
            RHS -= y[column]*A[row,column]
        y[row] = RHS
    #back solve
    for row in range(N-1,-1,-1):
        RHS = y[row]
        for column in range(row+1,N):
            RHS -= x[column]*A[row,column]
        x[row] = RHS/A[row,row]
    return x


Let's test on the same system as above.

In [22]:
#let's try it on a 3 x 3 to start
A = np.array([(3.0,2,1),(-1,4,5),(2,-8,10)])
Aorig = A.copy()
print("Originally A =\n",Aorig)
LU_factor(A)
print("\nAfter factorization A =\n",A)

b = np.array([6,8,4])
x = LU_solve(A,b)
print("\nThe solution to the system is",x)

Originally A =
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]]

After factorization A =
 [[ 3.          2.          1.        ]
 [-0.33333333  4.66666667  5.33333333]
 [ 0.66666667 -2.         20.        ]]

The solution to the system is [1. 1. 1.]


Let's resolve for $\vec{x} = [2, 12, 2019]^T$

In [23]:
solution2 = np.array([2,12,2019.])
b = np.dot(Aorig,solution2)
x = LU_solve(A,b)
print("The solution to the system is",x)

The solution to the system is [2.000e+00 1.200e+01 2.019e+03]


## LU with Pivoting

Pivoting with LU is needed in the same cases as it is in Gaussian elimination.  However, when we switch equations around, we need to make sure that we keep track of where we've switched rows, so we can do the same to the right-hand side when we do our forward and back solves. This complicates things considerably, but only because we have to add that bookkeeping to our algorithm.

First, we'll need to be able to swap rows in the matrix:

In [24]:
def swap_rows(A, a, b):
    """Rows to rows in a matrix, switch row a with row b
    args: 
    A: matrix to perform row swaps on
    a: row index of matrix
    b: row index of matrix
    
    returns: 
    nothing
    
    side effects:
    changes A to have rows a and b swapped
    """
    assert (a>=0) and (b>=0)
    N = A.shape[0] #number of rows
    assert (a<N) and (b<N) #less than because 0-based indexing
    temp = A[a,:].copy()
    A[a,:] = A[b,:].copy()
    A[b,:] = temp.copy()

Next, we'll change our algorithm to handle pivoting and keep track of the pivots:

In [25]:
def LU_factor_pivot(A,LOUD=True):
    """Factor in place A in L*U=A. 
    The lower triangular parts of A
    are the L matrix.  
    The L has implied ones on the diagonal.

    Args:
        A: N by N array
    Returns:
        a vector holding the order of the rows, 
        relative to the original order
    Side Effects:
        A is factored in place.
    """
    
    # Check dimensions
    [Nrow, Ncol] = A.shape
    assert Nrow == Ncol
    N = Nrow
    
    # Create scale factors 
    s = np.zeros(N)
    count = 0
    row_order = np.arange(N)
    for row in A:
        # Save largest absolute element in each row
        s[count] = np.max(np.fabs(row))
        count += 1
    if LOUD:
        print("s =",s)
    if LOUD:
        print("Original Matrix is\n",A)
    
    # loop over columns
    for column in range(0,N):
        #swap rows if needed
        largest_pos = np.argmax(np.fabs(A[column:N,column]/
                                        s[column])) + column
        if (largest_pos != column):
            if (LOUD):
                print("Swapping row",column,"with row",
                      largest_pos)
                print("Pre swap\n",A)
            swap_rows(A,column,largest_pos)
            #keep track of changes to RHS
            tmp = row_order[column]
            row_order[column] = row_order[largest_pos]
            row_order[largest_pos] = tmp
            #re-order s
            tmp = s[column]
            s[column] = s[largest_pos]
            s[largest_pos] = tmp
            if (LOUD):
                print("A =\n",A)
        for row in range(column+1,N):
            mod_row = A[row]
            factor = mod_row[column]/A[column,column]
            mod_row = mod_row - factor*A[column,:]
            #put the factor in the correct place in the modified row
            mod_row[column] = factor
            #only take the part of the modified row we need
            mod_row = mod_row[column:N]
            A[row,column:N] = mod_row
    return row_order

Now let's verify with a 4x4 system

In [26]:
#let's try it on a 4 x 4 to start
A = np.array([(3.0,2,1,-2),(-1,4,5,4),(2,-8,10,3),(-2,-8,10,0.1)])
print("A =\n",A,"\n")
answer = np.ones(4)
print("True solution. x =",answer,"\n")
b = np.dot(A,answer)
print("Thus b =",b,"\n")

# Apply LU with pivoting
row_order = LU_factor_pivot(A,False)

# Print A and the row order
print("After factorization A =\n",A,"\n")
print("The new row order is",row_order)

A =
 [[ 3.   2.   1.  -2. ]
 [-1.   4.   5.   4. ]
 [ 2.  -8.  10.   3. ]
 [-2.  -8.  10.   0.1]] 

True solution. x = [1. 1. 1. 1.] 

Thus b = [ 4.  12.   7.   0.1] 

After factorization A =
 [[ 3.          2.          1.         -2.        ]
 [ 0.66666667 -9.33333333  9.33333333  4.33333333]
 [-0.33333333 -0.5        10.          5.5       ]
 [-0.66666667  0.71428571  0.4        -6.52857143]] 

The new row order is [0 2 1 3]


We need to change the LU_solve function to take advantage of the swapped rows.

In [27]:
def LU_solve_pivot(A,b,row_order):
    """Take a LU factorized matrix and solve it for RHS b

    Args:
        A: N by N array that has been LU factored with
        assumed 1's on the diagonal of the L matrix
        b: N by 1 array of righthand side
        row_order:  list giving the re-ordered equations
        from the the LU factorization with pivoting
    Returns:
        x: N by 1 array of solutions
        
    Side Effect:
        Reorders b using row_order
    """
    [Nrow, Ncol] = A.shape
    assert Nrow == Ncol
    assert b.size == Ncol
    assert row_order.max() == Ncol-1
    N = Nrow
    
    #reorder the equations
    tmp = b.copy()
    for row in range(N):
        b[row_order[row]] = tmp[row]
    #b now has the correct order
    
    # We can either reuse our LU_solve code from above now that b has
    # been reorded or copy and paste
    return LU_solve(A,b)
    '''
    x = np.zeros(N)
    #temporary vector for L^-1 b
    y = np.zeros(N)
    #forward solve
    for row in range(N):
        RHS = b[row]
        for column in range(0,row):
            RHS -= y[column]*A[row,column]
        y[row] = RHS
    #back solve
    for row in range(N-1,-1,-1):
        RHS = y[row]
        for column in range(row+1,N):
            RHS -= x[column]*A[row,column]
        x[row] = RHS/A[row,row]
    return x
    '''


print("A =\n",A,"\n\nRow order =",row_order,"\n")
x = LU_solve_pivot(A,b,row_order)
print("The solution to the system is",x)

A =
 [[ 3.          2.          1.         -2.        ]
 [ 0.66666667 -9.33333333  9.33333333  4.33333333]
 [-0.33333333 -0.5        10.          5.5       ]
 [-0.66666667  0.71428571  0.4        -6.52857143]] 

Row order = [0 2 1 3] 

The solution to the system is [1. 1. 1. 1.]


<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
<b>Class Activity</b>: Verify that our LU decomposition with pivoting code works for other test cases.
</div>

In [28]:
epsilon = 1e-14
A_eps = np.array([(epsilon,-1,1),(-1,2,-1),(2,-1,0)])
print(A_eps)
b_eps = np.array([0,0,1.0])
print(b_eps)

Asave = A_eps.copy()
bsave = b_eps.copy()

[[ 1.e-14 -1.e+00  1.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 2.e+00 -1.e+00  0.e+00]]
[0. 0. 1.]


In [29]:
row_order = LU_factor_pivot(A_eps,False)
x_eps = LU_solve_pivot(A_eps,b_eps,row_order)

print("The solution is",x_eps)
print("The residual is",bsave - np.dot(Asave,x_eps))

The solution is [1. 1. 1.]
The residual is [0.00000000e+00 1.11022302e-16 0.00000000e+00]


## Errors in Linear Systems

We will now learn about condition number and singular value decomposition as tools to analyze solutions to linear systems. These are helpful for diagnosing if a linear system is rank deficient, i.e., the model is not fully specified (not enough equations or information given to solve).

### Rank of the Matrix

The rank of a matrix is the number of linear independent equations encoded in it. Let's explore a few examples.

#### Example 1

Let's take a look at the same $\mathbf{A}$ matrix we used as an example in this class and the previous class. We know this matrix corresponds to a system of three independent linear equations:

$$3 x_1 + 2 x_2 + x_3 = 6$$
$$-x_1 + 4 x_2 + 5x_3 = 8$$
$$2x_1 -8 x_2 + 10 x_3 = 4$$

$$\underbrace{\begin{pmatrix} 3 & 2 & 1\\ -1 & 4 & 5\\ 2& -8 & 10\end{pmatrix}}_{\mathbf{A}} ~
\underbrace{\begin{pmatrix} x_1\\x_2\\x_3\end{pmatrix}}_{\mathbf{x}} = \underbrace{\begin{pmatrix}6\\8\\4\end{pmatrix}}_{\mathbf{b}}
$$

Note: whether or not each row (equation) is independent does not depend on the right hand side. In other words, we are only focusing on $\mathbf{A}$.

In [30]:
A1 = np.array([(3.0,2,1),(-1,4,5),(2,-8,10)])
print("A = \n",A1)

A = 
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]]


In [31]:
print("rank(A) =",np.linalg.matrix_rank(A1))

rank(A) = 3


This makes sense. There are three linearly independent equations, so the rank of the matrix is 3.

#### Example 2

Now let's consider a system where equation 2 and 3 are the same:

$$1 x_1 + 1 x_2 + 1 x_3 = 0$$
$$2 x_1 + 1 x_2 + 0 x_3 = 0$$
$$2 x_1 + 1 x_2 + 0 x_3 = 0$$

Clearly, there are only two unique linear equations. Let's verify the rank is two.

In [32]:
A2 = np.array([[1, 1, 1], [2, 1, 0], [2, 1, 0]])
print("A2 = \n",A2)

print("\nrank(A2) =",np.linalg.matrix_rank(A2))

A2 = 
 [[1 1 1]
 [2 1 0]
 [2 1 0]]

rank(A2) = 2


#### Example 3

Let's consider a more subtle example.

$$1 x_1 + 0 x_2 + 0 x_3 + 1 x_4 = 1$$
$$0 x_1 + 1 x_2 + 0 x_3 + 1 x_4 = 2$$
$$0 x_1 + 0 x_2 + 1 x_3 - 1 x_4 = 3$$
$$2 x_1 + 2 x_2 + 2 x_3 + 2 x_4 = 4$$


<div style="background-color: rgba(0,255,0,0.05) ; padding: 10px; border: 1px solid darkgreen;"> 
    <b>Home Activity</b>: Convert the linear system above into a matrix. Store the left hand side in the numpy array (or matrix) <tt>A3</tt>. Then calculate the rank of <tt>A3</tt> and store in <tt>r3</tt>. Finally, write a sentence as to why the rank makes sense.
</div>

In [33]:
### BEGIN SOLUTION
A3 = np.array([[1, 0, 0, 1],[0, 1, 0, 1], [0, 0, 1, -1], [2, 2, 2, 2]])
print("A3 = \n",A3)
r3 = np.linalg.matrix_rank(A3)
print("\nrank(A3) =",r3)
### END SOLUTION

A3 = 
 [[ 1  0  0  1]
 [ 0  1  0  1]
 [ 0  0  1 -1]
 [ 2  2  2  2]]

rank(A3) = 3


In [34]:
### HIDDEN TESTS
A3_ans = np.array([[1, 0, 0, 1],[0, 1, 0, 1], [0, 0, 1, -1], [2, 2, 2, 2]])

assert np.shape(A3) == (4,4), "A3 should be a 4x4 matrix"

for i in range(4):
    for j in range(4):
        assert np.abs(A3_ans[i,j] - A3[i,j]) < 1E-6, "Check the ("+str(i)+","+str(j)+") element in A3"

### HIDDEN TESTS

Please see your linear algebra textbook for a more formal definition of rank. This wikipedia article is also quite good: https://en.wikipedia.org/wiki/Rank_(linear_algebra)

### Singular Value Decomposition

Singular Value Decomposition (SVD) is an incredibly important tool in numeric analysis and engineering. SVD is at the core of image compression, singular processing, control and many other important applications. There is a chapter on SVD in your linear algebra textbook. I encourage you to read it.

For our purposes, we will think of SVD as another type of matrix factorization. We can represent a $m \times n$ matrix as

$$\mathbf{A} = \mathbf{U} \cdot \mathbf{\Sigma} \cdot \mathbf{V}^T$$

where the columns of $\mathbf{U}$ and $\mathbf{V}$ are the left and right **singular vectors**:

$$\mathbf{U} = \begin{bmatrix} \vec{u}_1 & \vec{u}_2 & ... & \vec{u}_m \end{bmatrix}$$

$$\mathbf{V} = \begin{bmatrix} \vec{v}_1 & \vec{v}_2 & ... & \vec{v}_n \end{bmatrix}$$

Moreover, $\mathbf{\Sigma}$ is a $m \times n$ matrix of zeros with the singular values on the diagonal. For $m > n$, we have:

$$\mathbf{\Sigma} =
\begin{bmatrix} \sigma_1 & 0 & \dots & 0 \\ 
0 & \sigma_2 & \dots & 0 \\ 
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & \sigma_n \\
0 & 0 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 0
\end{bmatrix}$$

Thus, we can rewrite the SVD factorization as follows (assuming $m \geq n$):

$$\mathbf{A} = \sum_{i=1}^{n} \vec{u}_i \cdot \sigma_i \cdot \vec{v}_i^T$$

<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
    <b>Class Activity</b>: What are the dimensions of <b>A</b>, <b>U</b>, <b>V</b>, and $\mathbf{\Sigma}$? What changes if $n < m$?
</div>

Now let's look at SVD in Python:

In [35]:
A = np.array([(3.0,2,1),(-1,4,5),(2,-8,10),(-2,-8,10)])
print("A =\n",A,"\n")

U,S,V = np.linalg.svd(A)
print("U =\n",U,"\n")
print("S =\n",S,"\n")
print("V =\n",V,"\n")

A =
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]
 [-2. -8. 10.]] 

U =
 [[ 0.02399156 -0.33975594 -0.698885    0.62892771]
 [-0.08707759 -0.93759246  0.25472139 -0.2201247 ]
 [-0.70322927  0.04758521 -0.50005219 -0.50314217]
 [-0.70520244  0.056762    0.4434237   0.55031174]] 

S =
 [18.17773759  6.60309487  4.23898503] 

V =
 [[ 0.00896694  0.60332736 -0.79744322]
 [-0.01514879 -0.7973018  -0.60339071]
 [-0.99984504  0.01749087  0.00199033]] 



Notice that Python only returns the first $n$ elements of the diagonal $\mathbf{\Sigma}$. Because $m > n$, we know the remaining $m-n = 1$ elements are zero.

Let's check the rank of this matrix.

In [36]:
np.linalg.matrix_rank(A)

3

The matrix rank is 3 and $n=3$, thus we say this matrix is **full rank**. Notice SVD returned 3 non-zero singular values.

Now let's try SVD on a rank deficient matrix.

In [37]:
Anew = np.array([[1, 1, 0],[1, -1, 2],[2, 0, 2]])
print("Anew = \n",Anew,"\n")
print("and has rank",np.linalg.matrix_rank(Anew))

Anew = 
 [[ 1  1  0]
 [ 1 -1  2]
 [ 2  0  2]] 

and has rank 2


In [38]:
U,S,V = np.linalg.svd(Anew)
print("U =\n",U,"\n")
print("S =\n",S,"\n")
print("V =\n",V,"\n")

U =
 [[-0.13550992  0.8051731  -0.57735027]
 [-0.6295454  -0.51994159 -0.57735027]
 [-0.76505532  0.28523152  0.57735027]] 

S =
 [3.64575131e+00 1.64575131e+00 6.69376379e-17] 

V =
 [[-0.6295454   0.13550992 -0.76505532]
 [ 0.51994159  0.8051731  -0.28523152]
 [ 0.57735027 -0.57735027 -0.57735027]] 



Notice the almost zero third singular value. This matrix is numerically rank deficient. The number of non-zero singular values is the rank of a matrix.

### Condition Number

The condition number is the ratio of the largest to smallest singular values of a matrix:

$$\kappa = \frac{\sigma_{\mathrm{max}}}{\sigma_{\mathrm{min}}}$$

In some sense, the condition number $\kappa$ measures how close a matrix is to singular. The condition number also bounds the relative error in solving a linear system:

$$\frac{||\vec{\Delta x}||}{||\vec{x}||} \leq \kappa \frac{||\vec{\Delta b}||}{||\vec{b}||}$$

where $\vec{\Delta b}$ is a perturbation (error) in the vector $\vec{b}$ and $\vec{\Delta x}$ is the corresponding or induced error in $\vec{x}$. The operator $|| \cdot ||$ is the $\ell_2$ norm (i.e., length of a vector) you learned about in physics:

$$|| \vec{a} || = \sqrt{\sum_{i=1}^{N} a_i^2}$$

where $a$ is an arbitrary $N \times 1$ vector.

#### Example

Let's revisit the motivating example from last class.

$$3 x_1 + 2 x_2 + x_3 = 6$$
$$-x_1 + 4 x_2 + 5x_3 = 8$$
$$2x_1 -8 x_2 + 10 x_3 = 4$$

In [39]:
# Declare matrix
A33 = np.matrix([(3.0,2,1),(-1,4,5),(2,-8,10)])
b3 = np.matrix([6,8,4])

print("A = \n",A33)
print("\nb =",b3)

A = 
 [[ 3.  2.  1.]
 [-1.  4.  5.]
 [ 2. -8. 10.]]

b = [[6 8 4]]


In [40]:
## Calculate condition number
kappa = np.linalg.cond(A33)
print("condition number =",kappa)

condition number = 3.99784371803463


In [41]:
## Calculate the norm of a vector, such as b3
print("l2 norm of b =",np.linalg.norm(b3))

l2 norm of b = 10.770329614269007


<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
    <b>Class Activity</b>: Let's say in this linear system $\vec{b}$ comes from an experiment measurement with $\pm 0.1$ absolute error for each element, thus $\vec{\Delta b} = [0.1, 0.1, 0.1]^T$. What is the corresponding error when we solve the linear system for $\vec{x}$. In other words, what is $||\vec{\Delta x}||$?
</div>

In [42]:
### BEGIN SOLUTION
x = np.linalg.solve(A33,np.transpose(b3))
delta_b = np.array([0.1, 0.1, 0.1])
delta_b = np.transpose(delta_b)
delta_b_norm = np.linalg.norm(delta_b)
print(x)
print(np.linalg.norm(x))
print("Error in x is less than or equal to ",kappa*delta_b_norm/np.linalg.norm(b3) )
### END SOLUTION

[[1.]
 [1.]
 [1.]]
1.7320508075688772
Error in x is less than or equal to  0.06429207543641194


## Take Home Messages


1) LU factorization is *very* similar to Gaussian elimination:
 * $\mathbf{L}$ matrix saves the scaling factors from Gaussian elimination
 * $\mathbf{U}$ matrix is the same as the reduced matrix
 * $\mathbf{L}$ and $\mathbf{U}$ save all of the steps from matrix reduction, making LU factorization efficient to resolve $\mathbf{A} \vec{x} = \vec{b}$ for many different $\vec{b}$
 * Partial pivoting works the same way for both LU factorization and Gaussian elimination


2) Solving a linear system with LU factorization is easy:
 * First, perform forward substitution, which exploits that $\mathbf{L}$ is lower triangular
 * Second, perform backward substitution, which exploits that $\mathbf{U}$ is upper triangular



3) Rank and condition number tell us sensitivity of solutions to a linear system:
 * Rank tells us how many linear independent equations are encoded in a matrix
 * Singular value decomposition does many things... we'll use to tell if a matrix is *near-singular*, i.e., numerically rank deficient
 * Condition number tells us how $\vec{x}$, the solution to a linear system, changes with uncertainty in $\vec{b}$